<a href="https://colab.research.google.com/github/JulioLaz/telecom_churn/blob/row/telecom_churn_data_cleaning_jal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
global datos_churn

In [2]:
url='https://raw.githubusercontent.com/JulioLaz/telecom_churn/refs/heads/main/base_clientes.json'
def lectura_datos():
    global datos_churn
    response = requests.get(url)
    if response.status_code == 200:
        json_bruto = response.json()
        datos_churn = pd.json_normalize(json_bruto)
        return datos_churn
    else:
        print(f"Error: No se pudo acceder al archivo. Código de estado: {response.status_code}")
        return None

In [3]:
def tratamiento_datos():
    global datos_churn
    df= datos_churn.copy()
    df.drop('id_cliente', axis=1, inplace=True)
    df['cuenta.cobros.Total'] = df['cuenta.cobros.Total'].replace(' ', np.nan).astype(float)
    df = df[df['Churn'].isin(['si', 'no'])]

    # Calcular la moda para cuenta.contrato:
    contrato_mode = df['cuenta.contrato'].mode()[0]
    df.loc[:, 'cuenta.contrato'] = df['cuenta.contrato'].fillna(contrato_mode)

    # Calcular los cuartiles y el rango intercuartil y aplicar rango:
    Q1,Q3 = df['cliente.tiempo_servicio'].quantile(0.25),df['cliente.tiempo_servicio'].quantile(0.75)
    df=df[df['cliente.tiempo_servicio']<(Q3 + 1.5 * (Q3 - Q1))]
    df=df[df['cliente.tiempo_servicio']>=0]

    # Calcular la mediana de 'cuenta.cobros.mensual'
    median_cobros_mensual = df['cuenta.cobros.mensual'].median()
    df['cuenta.cobros.mensual'].fillna(median_cobros_mensual, inplace=True)

    # llenar col cuenta.cobros.Total con nulos:
    def calcular_cobros_total(row):
      if pd.isnull(row['cuenta.cobros.Total']):
        if row['cliente.tiempo_servicio'] == 0:
          return row['cuenta.cobros.mensual']
        else:
          return row['cuenta.cobros.mensual'] * row['cliente.tiempo_servicio']
      else:
        return row['cuenta.cobros.Total']
    df['cuenta.cobros.Total'] = df.apply(calcular_cobros_total, axis=1)

    # Calcular la moda de 'cuenta.facturacion_electronica'
    moda_facturacion_electronica = df['cuenta.facturacion_electronica'].mode()[0]
    df['cuenta.facturacion_electronica'].fillna(moda_facturacion_electronica, inplace=True)

    # Calcular la moda de 'cuenta.metodo_pago'
    moda_metodo_pago = df['cuenta.metodo_pago'].mode()[0]
    df['cuenta.metodo_pago'].fillna(moda_metodo_pago, inplace=True)

    # Mapear la columna 'churn' a valores booleanos:
    df['Churn'] = df['Churn'].map({'si': True, 'no': False})
    df['Churn'] = df['Churn'].astype(int)

    # Aplicar dummies a las col categoricas:
    categorical_columns = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

    # hacer discretas las columnas continuaas:
    columnas_discretas = ['cuenta.cobros.Total', 'cuenta.cobros.mensual']
    columnas_discretas_time = ['cliente.tiempo_servicio']
    est = KBinsDiscretizer(n_bins=40, encode='ordinal', strategy='quantile') #discretizar las columnas
    est_time = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile') #discretizar las columnas
    df[columnas_discretas] = est.fit_transform(df[columnas_discretas])
    df['cliente.tiempo_servicio'] = est_time.fit_transform(df[['cliente.tiempo_servicio']])
    datos_churn=df

In [7]:
def rf_model():
    x = datos_churn.drop('Churn', axis=1)
    y = datos_churn['Churn']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state = 50)
    model = RandomForestClassifier(random_state=50)
    model.fit(train_x, train_y)
    y_pred = model.predict(test_x)
    accuracy = accuracy_score(test_y, y_pred)
    print("Precisión:", round(accuracy,4))

In [8]:
lectura_datos()
tratamiento_datos()
rf_model()

Precisión: 0.8066
